*Dear Andrew: this console version is adapted from the program version. With pygame a lot of the times I cannot write functions with arguments (because the name of the function has to be an argument of another function, and in this case somehow pygame doesn't allow any arguments to be passed). As a results I had to use 'global' a lot instead of making my functions take arguments. So you'll see a lot of 'global's as well here, but it's just that I'm too dumb to adjust everything...*

# The CFG Maze Game

## What is the Maze task?

The Maze Task is an experimental task used by many psycholinguists. In this task, the participant follows a sentence through a "maze". They will see two words at a time, and one of the words will be the correct word that continues the sentence. The participant will only reach the end of the sentence by selecting the correct word at each step.

For example, for the sentence 'The rain fell silently', the participant may see something like this:

![Example maze task presentation from https://www.u.arizona.edu/~kforster/MAZE/how_it_works.htm](assets/example-maze-task.gif)

This task forces the reader into an incremental mode of processing in which each word must be fully integrated with the preceding context before the next word can be considered. Psycholinguists usually measure the reaction time needed to make the correct selection, which is associated with the difficulty of integrating the word with the previous words (the context).

## The Maze task as a game

Psycholinguists use the maze task to study human sentence processing, but as you can see, for participants in the maze task, there is a goal (to get to the correct sentence) and there is a challenge (to avoid the wrong choice), so it can be something like a game.

## A CFG Maze game

We just learned how to use CFG to generate sentences in Python. We can use that to generate trials in the Maze task. Because the sentences we generate from CFG don't have the 'points of processing difficulty' that psycholinguists study, so we can simply make it a small game (instead of a proper experiment that answers a research question).

### Modify generate() to return both the sentence and the alternative option at each position

Remember in a maze task, at each word, the participant sees two options, one is correct, the other is not. We can modify generate() to give us both the sentence (the correct options) and the alternatives (the incorrect options).

First let's consider what should be the alternative (incorrect) option? It has to be incorrect, so let's just say in the context of CFG, it cannot be a grammatical continuation of the last word. This means that the correct option needs to block all words that belong to the same node as the correct option, or any nodes that occupy the same syntactic position as the correct option's node. This is what's in `block_list`.

In [1]:
import random

# our cfg dictionary
cfg = {"NP":[["D","N"],"cats","dogs"],"VP":[["V","NP"],"walk","sleep"],"S":[["NP","VP"]],"D":["the","a"],"N":["cat","dog"],"V":["love","tolerate"]}

# block_list has nodes as keys, and as values, a list of nodes that are identical to the key or occupy the same syntactic position as the key
block_list = {'NP':['NP','N','D'], 'N':['N','NP'], 'VP':['VP', 'V'], 'V':['V', 'VP'], 'D':['D']}

Next, let's modify the generate() function. We make it return a list of [correct, wrong, correct, wrong] for the randomly generated sentence.

In [2]:
def generate(cfg, node='S'):
    expansion = random.choice(cfg[node])
    if type(expansion) == list:
        return generate(cfg, expansion[0]) + generate(cfg, expansion[1])
    elif type(expansion) == str:
        # block the node that makes sense after the current node
        blocked_nodes = block_list[node]
        # get a dictionary of nodes:words that are not in blocked nodes
        option_pool = {key:cfg[key] for key in [i for i in list(cfg.keys()) if i not in blocked_nodes]}
        # get a list of options to choose from from the values of option_pool that are strings, but only if the element is a string (only if it's a word, not a list of nodes)
        options = [x for lis in list(option_pool.values()) for x in lis if type(x) == str]
        option = random.choice(options)
        return [expansion, option]

### Make the game

Now we can make the game!

Think about what the game does. First it should randomly generate a sentence, and print the first word as well as two options for the second word. Then the game needs to ask the player for a selection, and check whether that selection is correct. If the selection is correct, the game moves on to the next word, such that the first two words are printed, and the options for the third word is shown.... If the selection is wrong, the player has lost this round. If the player gets to the end of the sentence making all correct choices, then the player wins.

For me, I've broken down what the game needs to do into a few different functions: a function that generates the sentence, one that prints the current sentence and the options, one that asks the player for a selection, and one main game function that brings everything together.

Let's call the first function getSentence(). This needs to generate a sentence using generate(), and we also ask it to parse the list returned by generate() so it looks nicer.

In [3]:
def getSentence():
    # global sents the variables to the global environment (outside the function), because we'll need these in other functions as well
    # sentence records the generated sentence
    # selected records what correct choices the player has selected
    # current_location records which words we're at in the sentence, in this function we just initialise it.
    global sentence, selected, current_location
    # get a flat sentence list first
    sentence_flat = generate(cfg)
    # then parse the flat sentence into a dictionary of location:options
    sentence = {i:[sentence_flat[i*2], sentence_flat[(i*2)+1]] for i in range(int(len(sentence_flat)/2))}
    # record the first correct word (which we just show the player) in selected
    selected = [sentence[0][0]]
    current_location = 1
    # # then get the options for the current location in the sentence
    # options = scramble_options(sentence, current_location)



Next let's call this function printSentence(). It needs to do a few things:
- print the words selected by the player that's correct.
- get the options and scramble the options using scramble_options().
- print the options.

In [4]:
# First we need a function to scramble the options (so that the correct option is not always the left one...)
def scramble_options(sentence, current_location):
    # get options
    options = sentence[current_location].copy()
    # scramble the order of correct/incorrect options
    random.shuffle(options)
    return options

def printSentence():
    global sentence, options, current_location
    print('----------')
    # print the (incomplete) sentence
    print(' '.join(selected))
    # get new options
    options = scramble_options(sentence, current_location)
    # print the options
    print('  '+options[0]+'        '+options[1])

We need a function to allow the player to make a choice. Let's call this getChoice(). This function simply returns a number (1 for the left word, 2 for the right word), and we'll check whether the choice is correct later.

In [5]:
def getChoice():
  while True:
    try:
      choice = int(input('Which one do you choose?'))
      if choice in [1,2]:
        return choice
      else:
        print('Enter 1 for the left option, 2 for the right option.')
    except ValueError:
      print('Enter 1 for the left option, 2 for the right option.')

Now we have everything we need and we can work on the main game function. Here's what it does:

- first print some welcome instructions.
- let's make one game have five sentences. So for trial numbers 1 to 5:
    - get a randomly generated sentence and print it
    - for each word in that sentence:
        - get the players choice
        - check if the players choice is correct.
            - if so, add the correct choice to the presentation of sentences and present options for the next word; and if we've reached the end of the sentence, print congratulations and print the player's score.
            - if not, end the trial immediately after printing a message and the player's score.
- when all 5 trials have finished, print the player's score and exit.

In [6]:
def main():
    global sentence, options, current_location
    print('Welcome to the CFG Maze game!')
    print('Please select the option that makes the sentence grammatical.')
    print('Enter 1 for the left option, 2 for the right option.')
    correct_count = 0
    for i in range(5):
        getSentence()
        printSentence()
        current_location = 1
        for j in range(len(sentence)):
            choice = getChoice()
            # if correct choice
            if options[choice-1] == sentence[current_location][0]:
                # if we're at the end of a sentence
                if current_location == len(sentence)-1:
                    selected.append(sentence[current_location][0])
                    correct_count += 1
                    print('----------')
                    # print the correct sentence
                    print(' '.join([sentence[i][0] for i in range(len(sentence))]))
                    print('Congrats!')
                    print('Your score: '+str(correct_count)+'/5')
                    break
                else:
                    selected.append(sentence[current_location][0])
                    current_location += 1
                    printSentence()
            # if not correct choice
            else:
                print('----------')
                print('Wrong...')
                print('Answer: '+' '.join([sentence[i][0] for i in range(len(sentence))]))
                print('Your score: '+str(correct_count)+'/5')
                break
    print('Game end!')
    print('Your score: '+str(correct_count)+'/5')

The following code allows main() to be run when you execute the .py file as a script.

In [8]:
if __name__ == "__main__":
    main()

Welcome to the CFG Maze game!
Please select the option that makes the sentence grammatical.
Enter 1 for the left option, 2 for the right option.
----------
dogs
  the        walk
Which one do you choose?2
----------
dogs walk
Congrats!
Your score: 1/5
----------
the
  dog        a
Which one do you choose?1
----------
the dog
  cat        tolerate
Which one do you choose?1
----------
Wrong...
Answer: the dog tolerate cats
Your score: 1/5
----------
the
  cat        sleep
Which one do you choose?1
----------
the cat
  dogs        walk
Which one do you choose?2
----------
the cat walk
Congrats!
Your score: 2/5
----------
cats
  love        cat
Which one do you choose?1
----------
cats love
  the        walk
Which one do you choose?1
----------
cats love the
  cat        walk
Which one do you choose?1
----------
cats love the cat
Congrats!
Your score: 3/5
----------
a
  dog        the
Which one do you choose?2
----------
Wrong...
Answer: a dog sleep
Your score: 3/5
Game end!
Your score: 3/